# PHD - DISC24 - Exp 1 - KL for full circuits - v1 - 20240406

//======================================================================================================
// 

EPUSP - Quantum circuits dimensions - v.03.2024 
Author: Waldemir Cambiucci 
Data: April 6th, 2024

File: PHD - DISC24 - Exp 1 - KL for full circuits - v1 - 20240406

Description: 

https://www.calculatorsoup.com/calculators/discretemathematics/combinations.php
https://pnnl.github.io/hypernetx-widget/

//
//======================================================================================================

In [1]:
import re
import networkx as nx
from qiskit import QuantumCircuit
from networkx.algorithms import community

In [2]:
def extract_qubits(input_str):
    # Regular expression to match qubit numbers
    pattern = r'q\[(\d+)\]'
    # Find all matches of the pattern in the input string
    matches = re.findall(pattern, input_str)
    # Extract qubit numbers from matches
    qubit_numbers = [int(match) for match in matches]
    # Return the qubit numbers as separate variables
    if len(qubit_numbers) >= 2:
        return tuple(qubit_numbers)
    elif len(qubit_numbers) == 1:
        return qubit_numbers[0], None
    else:
        return None, None

In [3]:
def get_num_qubits_from_qasm(qasm_str):  
    for line in qasm_str.splitlines():
        if line.startswith('qreg'):        
            pos = line.find('[') + 1
            end_pos = line.find(']')
            qubits_str = line[pos:end_pos]    
    return int(qubits_str)

In [4]:
def convert_qasm_in_hypergraph_kl(qasm_str):     
    g = nx.Graph()       
    nqubits = get_num_qubits_from_qasm(qasm_str)
    for line in qasm_str.splitlines():                
        if line.startswith('cx') or line.startswith('cz'):            
            qubit1, qubit2 = extract_qubits(line)            
            g.add_edge(qubit1,qubit2)            
    return g           

In [5]:
def count_ebits(g , b):      
    nebits = 0
    for edge in g.edges:
        n1 = edge[0]
        n2 = edge[1]        
        if (n1 in b[0]) and (n2 in b[1]):   
            nebits = nebits + 1
        elif (n1 in b[1]) and (n2 in b[0]):   
            nebits = nebits + 1
    return nebits

In [6]:
def get_coupling_base_from_circuit(circuit):
    n = circuit.num_qubits
    Cb = (n * (n-1))/2  
    return Cb

In [7]:
def get_coupling_ratio_from_circuit(circuit):
    num_bin_gates = count_binary_operations(circuit)
    Cb = get_coupling_base_from_circuit(circuit)
    Cr = num_bin_gates / Cb
    return Cr

In [8]:
def count_binary_operations(quantum_circuit):
    # Count the operations using count_ops() function
    ops_dict = quantum_circuit.count_ops()    
    # Sum the counts of binary operations
    total_binary_ops = sum(count for op, count in ops_dict.items() if op in ['cx', 'cz', 'swap', 'cp'])    
    return total_binary_ops

In [17]:
def create_full_circuit(n):
    bc = QuantumCircuit(n)    
    total_unique_gates = (n*(n-1))/2     
    for q0 in range(0,n):
        bc.barrier()
        for q1 in range(q0+1,n):
            bc.cz(q0,q1)    
    return bc

In [73]:
def split_circuit_and_count_binary_gates(circuit):
    # Get the number of qubits in the circuit
    num_qubits = circuit.num_qubits
    
    # Calculate the midpoint to split the circuit
    midpoint = num_qubits // 2
    
    # Divide the qubits into two partitions
    partition_1 = list(range(midpoint))    
    partition_2 = list(range(midpoint, num_qubits))
    
    # Initialize count for binary gates involving qubits from both partitions
    count_binary_gates = 0
    
    # Iterate through the gates in the circuit
    for instr, qargs, cargs in circuit.data:
        # Check if the gate is a binary gate and involves qubits from both partitions
        if len(qargs) == 2 and (qargs[0].index in partition_1 and qargs[1].index in partition_2) or \
           (qargs[0].index in partition_2 and qargs[1].index in partition_1):
            count_binary_gates += 1
    
    return count_binary_gates

# Testing functions with big circuits

In [27]:
test_circuit = create_full_circuit(4)

In [28]:
test_circuit.draw()

░           ░        ░     ░ 
q_0: ─░──■──■──■──░────────░─────░─
      ░  │  │  │  ░        ░     ░ 
q_1: ─░──■──┼──┼──░──■──■──░─────░─
      ░     │  │  ░  │  │  ░     ░ 
q_2: ─░─────■──┼──░──■──┼──░──■──░─
      ░        │  ░     │  ░  │  ░ 
q_3: ─░────────■──░─────■──░──■──░─
      ░           ░        ░     ░

In [29]:
qasm_str = test_circuit.qasm()
qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

In [30]:
qasm_hypergraph.nodes

NodeView((0, 1, 2, 3))

In [31]:
print(qasm_hypergraph.edges)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [32]:
blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=50)
print(blocks)

({2, 3}, {0, 1})


In [16]:
num_ebits = count_ebits(qasm_hypergraph,blocks)
print('total ebits with KL bipartition: ',num_ebits)

total ebits with KL bipartition:  4


In [36]:
get_num_ebits_mid_cut(test_circuit)

0

# DISC24 - Exp 1 - KL for full circuits

In [82]:
from datetime import datetime
def generate_timestamp():
    """
    Generates a timestamp string representing the current date and time.    
    Returns:
        str: Timestamp string in the format 'YYYY-MM-DD_HH-MM-SS'.
    """
    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')
    return timestamp

In [87]:
def disc_exp1_fullcircuits(min_qubits, max_qubits):
    
    timestamp = generate_timestamp()
    print("Timestamp:", timestamp)

    DISC_Experiment_folder = 'disc_exp1_full_circuits'

    print('Running experiment with FULL CIRCUITS - calculating w, s, Cb, min.cut, and segments using KL partitioning.')
    print('----------------------------------------------------------------------------------------------------------')
    print('...')

    # Write the contents of the list to a file named 'coupling.log'
    file1 = open(DISC_Experiment_folder + "\\" + 'disc_exp1_' + timestamp + '.log', 'w')
    file1.writelines('width (n)' + '\t' + 'depth (d)' + '\t' + 'size (s)' + '\t' + 'coupling (Cb)' + '\t' + 'mid.cut' + '\t' + 'min.cut KL'+ '\t' + 'bipartite segments' + '\n')

    #=====================================================================
    #
    for n in range(min_qubits,max_qubits):

        print('Full circuit for ' + str(n) + ' qubits.')

        circuit = create_full_circuit(n)

        d = circuit.depth()
        s = circuit.size()       
        mid_cuts = split_circuit_and_count_binary_gates(circuit)
        Cb = get_coupling_base_from_circuit(circuit)    
        Rc = get_coupling_ratio_from_circuit(circuit)        
        qasm_str = circuit.qasm()
        qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

        blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=200)
        num_ebits = count_ebits(qasm_hypergraph,blocks)

        file1.writelines( str(n) + '\t' + str(d) + '\t' + str(s) + '\t' + str(Cb) + '\t' + str(mid_cuts) + '\t' + str(num_ebits) + '\t' + str(blocks) + '\n' )

        #
        #=====================================================================

    print('...')
    print('----------------------------------------------------------------------------------------------------------')
    print('File closed.')

    file1.close()


In [88]:
disc_exp1_fullcircuits(4,121)

Timestamp: 2024-04-06_11-01-52
Running experiment with FULL CIRCUITS - calculating w, s, Cb, min.cut, and segments using KL partitioning.
----------------------------------------------------------------------------------------------------------
...
Full circuit for 4 qubits.
Full circuit for 5 qubits.
Full circuit for 6 qubits.
Full circuit for 7 qubits.
Full circuit for 8 qubits.
Full circuit for 9 qubits.
Full circuit for 10 qubits.
Full circuit for 11 qubits.
Full circuit for 12 qubits.
Full circuit for 13 qubits.
Full circuit for 14 qubits.
Full circuit for 15 qubits.
Full circuit for 16 qubits.
Full circuit for 17 qubits.
Full circuit for 18 qubits.
Full circuit for 19 qubits.
Full circuit for 20 qubits.
Full circuit for 21 qubits.
Full circuit for 22 qubits.
Full circuit for 23 qubits.
Full circuit for 24 qubits.
Full circuit for 25 qubits.
Full circuit for 26 qubits.
Full circuit for 27 qubits.


C:\Users\wcamb\AppData\Local\Temp\ipykernel_8880\1070940826.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  (qargs[0].index in partition_2 and qargs[1].index in partition_1):
C:\Users\wcamb\AppData\Local\Temp\ipykernel_8880\1070940826.py:18: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  if len(qargs) == 2 and (qargs[0].index in partition_1 and qargs[1].index in partition_2) or \


Full circuit for 28 qubits.
Full circuit for 29 qubits.
Full circuit for 30 qubits.
Full circuit for 31 qubits.
Full circuit for 32 qubits.
Full circuit for 33 qubits.
Full circuit for 34 qubits.
Full circuit for 35 qubits.
Full circuit for 36 qubits.
Full circuit for 37 qubits.
Full circuit for 38 qubits.
Full circuit for 39 qubits.
Full circuit for 40 qubits.
Full circuit for 41 qubits.
Full circuit for 42 qubits.
Full circuit for 43 qubits.
Full circuit for 44 qubits.
Full circuit for 45 qubits.
Full circuit for 46 qubits.
Full circuit for 47 qubits.
Full circuit for 48 qubits.
Full circuit for 49 qubits.
Full circuit for 50 qubits.
Full circuit for 51 qubits.
Full circuit for 52 qubits.
Full circuit for 53 qubits.
Full circuit for 54 qubits.
Full circuit for 55 qubits.
Full circuit for 56 qubits.
Full circuit for 57 qubits.
Full circuit for 58 qubits.
Full circuit for 59 qubits.
Full circuit for 60 qubits.
Full circuit for 61 qubits.
Full circuit for 62 qubits.
Full circuit for 63 